In [1]:
import cv2
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import shutil

from tqdm import tqdm

In [2]:
from tactile_learning.datasets.preprocess import dump_video_to_images, get_closest_id
from tactile_learning.utils.visualization import plot_tactile_sensor

In [3]:
import decimal

# Get the metadata of images and tactile information
def get_desired_indices(root, fps, view_num=0): # frames per second from the video to receive
    image_metadata_path = os.path.join(root, f'cam_{view_num}_rgb_video.metadata')
    tactile_info_path = os.path.join(root, 'touch_sensor_values.h5')

    with open(image_metadata_path, 'rb') as f:
        image_metadata = pickle.load(f)
        image_timestamps_array = np.asarray(image_metadata['timestamps'])
        image_timestamps = np.asarray(image_metadata['timestamps']) / 1000.
    with h5py.File(tactile_info_path, 'r') as f:
        tactile_timestamps = f['timestamps'][()]

    image_id, tactile_id = 0, 0
    curr_timestamp = tactile_timestamps[0] # These timestamps are in seconds
    image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

    tactile_indices, image_indices = [], []
    tactile_indices.append(tactile_id)
    image_indices.append(image_id)

    frame_period = 1. / fps
    while(True):
        curr_timestamp += frame_period
        tactile_id = get_closest_id(tactile_id, curr_timestamp, tactile_timestamps)
        image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

        if curr_timestamp > tactile_timestamps[tactile_id] and curr_timestamp > image_timestamps[image_id]:
            break

        tactile_indices.append(tactile_id)
        image_indices.append(image_id)

    assert len(tactile_indices) == len(image_indices)
    return tactile_indices, image_indices


In [4]:

def dump_states(root, tactile_indices, image_indices, view_num=0):
    # Make directory to dump the visualization
    pbar = tqdm(total=len(tactile_indices))

    with h5py.File(os.path.join(root, 'touch_sensor_values.h5'), 'r') as f:
        all_tactile_values = f['sensor_values'][()]

    viz_dir = os.path.join(root, 'visualization')
    os.makedirs(viz_dir, exist_ok=True)
    
    video_path = os.path.join(root, f'cam_{view_num}_rgb_video.avi')
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_id = 0
    for i in range(len(tactile_indices)):
        tactile_id, image_id = tactile_indices[i], image_indices[i]
        while frame_id != image_id and success:
            # Find the frame that is equal to image_id
            success, image = vidcap.read()
            frame_id += 1
        dump_demo_state(
            frame_id = i,
            viz_dir = viz_dir,
            tactile_values = all_tactile_values[tactile_id,:,:,:],
            camera_img = image
        )

        pbar.update(1)

    pbar.close()

def dump_demo_state(frame_id, viz_dir, tactile_values, camera_img):
    # tactile_values: (15,16,3)
    fig, axs = plt.subplots(figsize=(20,20), nrows=4, ncols=4)
    for row_id in range(4):
        for column_id in range(4):
            if row_id + column_id > 0: # The top left axis should stay empty
                plot_tactile_sensor(
                    ax = axs[column_id][row_id],
                    sensor_values = tactile_values[row_id*4 + column_id-1],
                    title = f'Sensor {row_id*4+column_id-1}'
                )
    fig.suptitle('Tactile State')
    fig.savefig(os.path.join(viz_dir, 'Tactile State.png'))
    fig.clf()
    plt.close()

    tactile_img = cv2.imread(os.path.join(viz_dir, 'Tactile State.png'))
    height_scale = camera_img.shape[0] / tactile_img.shape[0]
    tactile_img = cv2.resize(
        tactile_img,
        (int(tactile_img.shape[1] * height_scale),
         int(tactile_img.shape[0] * height_scale))
    )
    total_img = cv2.hconcat([camera_img, tactile_img])

    img_name = 'state_{}.png'.format(str(frame_id).zfill(3))
    cv2.imwrite(os.path.join(viz_dir, img_name), total_img)



In [6]:
import glob
# roots = [
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/bowl_picking/eval/demonstration_7',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/bowl_picking/eval/demonstration_9',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/bowl_picking/eval/demonstration_15',
# ]
# data_path = '/home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval'
data_path = '/data/tactile_learning/deployment_data/data/bowl_picking/demonstrations/image_generalization'
roots = glob.glob(f'{data_path}/demonstration_*')

retrieval_fps = 1
view_num=0
for root in roots:
    # print('root: {}'.format(root))
    tactile_indices, image_indices = get_desired_indices(root=root, fps=retrieval_fps, view_num=view_num)
    print('tactile_indices: {}, image_indices: {}'.format(tactile_indices, image_indices))
    dump_states(root, tactile_indices, image_indices, view_num=view_num)


tactile_indices: [0, 99, 197, 297, 395, 494, 593, 698, 790, 889, 986, 1084, 1183, 1282, 1380, 1478, 1577, 1674, 1773, 1873, 1971, 2068, 2178, 2267], image_indices: [70, 100, 130, 160, 190, 218, 248, 278, 308, 337, 367, 397, 426, 456, 485, 515, 545, 575, 605, 636, 666, 696, 726, 754]


100%|███████████████████████████████████████████████████████| 24/24 [00:26<00:00,  1.09s/it]


tactile_indices: [0, 99, 198, 296, 397, 492, 592, 698, 788, 888, 986, 1084, 1180, 1278, 1376, 1476, 1574, 1672, 1771, 1870, 1968, 2072, 2169, 2264, 2362, 2462, 2561, 2659, 2758, 2855, 2953, 3053, 3150, 3248, 3347, 3446, 3544, 3644, 3741, 3838, 3938, 4035, 4134, 4232, 4329, 4428, 4527, 4625, 4722, 4828, 4922, 5024, 5120, 5216, 5314, 5413, 5510, 5609, 5707, 5806, 5904, 6002, 6101, 6198, 6296, 6398, 6493, 6595, 6691, 6791, 6888, 6988, 7088, 7183, 7282, 7381, 7478, 7577, 7676, 7774, 7872, 7972, 8069, 8168, 8266, 8364, 8462, 8561, 8660, 8757, 8855, 8953, 9052, 9151, 9249, 9349, 9451, 9545, 9643, 9742, 9840, 9938, 10036, 10135, 10233, 10331, 10431, 10528, 10628, 10734, 10828, 10923, 11021, 11119, 11221, 11317, 11415, 11513, 11611, 11709, 11808, 11905, 12004], image_indices: [71, 100, 130, 160, 190, 220, 250, 280, 309, 339, 369, 399, 429, 459, 489, 519, 548, 578, 608, 638, 668, 698, 727, 757, 787, 817, 846, 876, 906, 936, 966, 996, 1024, 1054, 1083, 1113, 1143, 1173, 1203, 1233, 1263, 1292, 1

100%|█████████████████████████████████████████████████████| 123/123 [02:14<00:00,  1.09s/it]


tactile_indices: [0, 88, 186, 285, 383, 481, 579, 677, 775, 874, 974, 1071, 1170, 1278, 1375, 1466, 1573, 1663, 1762, 1860, 1959, 2055, 2155, 2253, 2351, 2454, 2547, 2648, 2751, 2849, 2944, 3048, 3140, 3237, 3336, 3434, 3533, 3631, 3730, 3828, 3927, 4025, 4122, 4221, 4319, 4419, 4518, 4619, 4716, 4813, 4914, 5010, 5108, 5206, 5304, 5402, 5501, 5599, 5698, 5796, 5895, 5994, 6091, 6188, 6287, 6385, 6484, 6583, 6681, 6780, 6879, 6976, 7074, 7175, 7270, 7372, 7468, 7566, 7666, 7764, 7862, 7961, 8059, 8158, 8256, 8355, 8454, 8553, 8651, 8750, 8848, 8947, 9044, 9143, 9241, 9339, 9438, 9536, 9634, 9734, 9831, 9930, 10028, 10128, 10225, 10323, 10422, 10520, 10619, 10718, 10816, 10913, 11011, 11109, 11208, 11306, 11405, 11503, 11604, 11701, 11800, 11903, 11997, 12095, 12193, 12291, 12390, 12488, 12587, 12685, 12784, 12883, 12981, 13079, 13187, 13281, 13376, 13473, 13572, 13669, 13768, 13866, 13964, 14063, 14161, 14260, 14357, 14456, 14555, 14659, 14762, 14859, 14955, 15049, 15147, 15246, 15344,

100%|█████████████████████████████████████████████████████| 207/207 [03:56<00:00,  1.14s/it]


KeyError: "Unable to open object (object 'timestamps' doesn't exist)"

In [7]:
print(roots)

['/home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demonstration_large', '/home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demonstration_largest_size', '/home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demonstration_raspberry_whole', '/home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demonstration_small_mid_size', '/home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demonstration_small_size', '/home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demonstration_mid_size', '/home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demonstration_smallest_size', '/home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demonstration_large_mid_size']


In [8]:
# Turn the images to a video and delete the directory
video_fps = 10
for root in roots:
    print('dumping video in root: {}'.format(root))
    video_path = os.path.join(root, f'visualization_{view_num}.mp4')
    if os.path.exists(video_path):
        os.remove(video_path)
    viz_dir = os.path.join(root, 'visualization')
    os.system('ffmpeg -r {} -i {}/%*.png -vf scale=2000x720,setsar=1:1 {}'.format(
        video_fps, # fps
        viz_dir,
        video_path
    ))


dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demonstration_large


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-18)
  configuration: --prefix=/home/irmak/miniconda3/envs/tactile_learning --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demonstration_largest_size


[libx264 @ 0x5559ad73c880] using SAR=1/1
[libx264 @ 0x5559ad73c880] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x5559ad73c880] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x5559ad73c880] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demons

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demonstration_raspberry_whole


...

[libx264 @ 0x560fc8c56f80] using SAR=1/1
[libx264 @ 0x560fc8c56f80] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x560fc8c56f80] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x560fc8c56f80] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demons

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demonstration_smallest_size


[libx264 @ 0x563d19894140] using SAR=1/1
[libx264 @ 0x563d19894140] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x563d19894140] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x563d19894140] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demons

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demonstration_large_mid_size


[libx264 @ 0x56149d15b380] using SAR=1/1
[libx264 @ 0x56149d15b380] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x56149d15b380] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x56149d15b380] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/tofu_picking/eval/demons

In [18]:
# Remove the img directory

for root in roots:
    viz_dir = os.path.join(root, 'visualization')
    shutil.rmtree(viz_dir)
